In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

In [2]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [3]:
loader = WebBaseLoader(
    web_paths=("https://www.abbynews.com/local-news/cleanup-crews-at-peardonville-road-encampment-in-abbotsford-7758944",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["details-body clearfix", "title details-title"]},
        )
    ),
)

docs = loader.load()
print(f"document numbers: {len(docs)}")
docs

document numbers: 1


[Document(metadata={'source': 'https://www.abbynews.com/local-news/cleanup-crews-at-peardonville-road-encampment-in-abbotsford-7758944'}, page_content='\nCleanup work took place Tuesday (Jan. 14) at the Peardonville Road encampment in Abbotsford.\nWork crews, including an excavator, were on hand to clear away garbage, but none of the occupants were being asked to leave. They were given prior notice about the cleanup.\nThe land, located in the area of South Fraser Way and Peardonville Road, is owned by the Ministry of Transportation and Infrastructure.\nA previous cleanup of the site was done in December by about 50 people from the street community. The project was organized by the Business Engagement and Ambassador Program.\nOther local camps on ministry land include ones at Whatcom Road and Cole Road.\nA spokesperson with the ministry said there will be ongoing cleanups of the Peardonville camp, as well as other ones along Highway 1 "to improve safety and cleanliness."\n"Everyone dese

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

3

In [5]:
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """You are a friendly AI assistant performing question-answering. 
    Your task is to answer the given question using the provided context.
If you cannot find the answer in the given context, or if you do not know the answer, respond with 
`The information to answer the question is not available in the given context`.
Please answer in English. However, do not translate technical terms or names.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)